## Libraries

In [1]:
from torch import nn
import torch.nn.functional as F
import torch
from torch.utils.data import DataLoader, Subset
from torchvision import transforms, datasets
from torchvision.utils import save_image
from tqdm import tqdm
import numpy as np

## VAE Architecture

In [2]:
class VariationalAutoEncoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=200, z_dim=20):
        super().__init__()
        # encoding
        self.img2hidden = nn.Linear(input_dim, hidden_dim)
        self.hidden2hidden = nn.Linear(hidden_dim, hidden_dim)
        self.hidden2mu = nn.Linear(hidden_dim, z_dim)
        self.hidden2sigma = nn.Linear(hidden_dim, z_dim)

        # decoder
        self.z2hidden = nn.Linear(z_dim, hidden_dim)
        self.hidden2image = nn.Linear(hidden_dim, input_dim)

        # define ReLU
        self.relu = nn.ReLU()

    def encoder(self, x):
        h = self.relu(self.img2hidden(x))
        h = self.relu(self.hidden2hidden(h))
        mu = self.hidden2mu(h)
        logvar = self.hidden2sigma(h)
        return mu, logvar

    def decoder(self, z):
        h = self.relu(self.z2hidden(z))
        h = self.relu(self.hidden2hidden(h))
        img = self.hidden2image(h)
        img = torch.sigmoid(img)
        return img


    def forward(self, x):
        mu, logvar = self.encoder(x)
        sigma = torch.exp(0.5 * logvar)
        epsilon = torch.randn_like(sigma)
        z_reparameterized = mu + sigma * epsilon

        x_reconstructed = self.decoder(z_reparameterized)

        return x_reconstructed, mu, logvar




## Train

In [3]:
device = torch.device("cpu")
input_dim = 784
hidden_dim = 200
z_dim = 20
num_epochs = 60
batch_size = 128
karpathy_constant = 3e-4



dataset = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
random_indices = np.random.choice(len(dataset), size=5000, replace=False)
subset_train_dataset = Subset(dataset, random_indices)

train_loader = DataLoader(
    dataset=subset_train_dataset,batch_size=batch_size,shuffle = True
    )

model = VariationalAutoEncoder(input_dim=input_dim,
                               hidden_dim=hidden_dim,
                               z_dim=z_dim)

model = model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr = karpathy_constant)
loss_fn = nn.BCELoss(reduction='sum')


for epoch in range(num_epochs):
    loop = tqdm(enumerate(train_loader), total=len(train_loader))

    epoch_loss = 0

    # not using labels for now
    for i, (x, _) in loop:
        x = x.to(device).view(x.shape[0], input_dim)

        x_reconstructed, mu, logvar = model(x)

        recontruction_loss = loss_fn(x_reconstructed, x)

        # kl divergence
        # minimizing the same as negative
        # pushes towards gaussian
        kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

        loss = recontruction_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        loop.set_postfix(loss=loss.item())

    avg_epoch_loss = epoch_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch+1}/{num_epochs}], Average Loss: {avg_epoch_loss:.2f}")


# save the model
torch.save(model.state_dict(), "vae_model.pth")

100%|██████████| 26.4M/26.4M [00:02<00:00, 12.7MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 201kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.78MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 22.1MB/s]
100%|██████████| 40/40 [00:01<00:00, 36.42it/s, loss=3.28e+3]


Epoch [1/60], Average Loss: 487.13


100%|██████████| 40/40 [00:00<00:00, 41.55it/s, loss=3.16e+3]


Epoch [2/60], Average Loss: 388.23


100%|██████████| 40/40 [00:00<00:00, 42.17it/s, loss=2.47e+3]


...

Epoch [58/60], Average Loss: 254.48


100%|██████████| 40/40 [00:01<00:00, 39.70it/s, loss=1.87e+3]


Epoch [59/60], Average Loss: 254.23


100%|██████████| 40/40 [00:00<00:00, 41.14it/s, loss=2.1e+3]

Epoch [60/60], Average Loss: 253.93


## Inference

In [4]:
# Load the trained model
model = VariationalAutoEncoder(input_dim=input_dim, hidden_dim=hidden_dim, z_dim=z_dim).to(device)
model.load_state_dict(torch.load("vae_model.pth"))  # Make sure to save your model at training time
model.eval()

# Load MNIST test data
test_dataset = datasets.FashionMNIST(root=".data/", train=False, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

# Get one batch
test_batch = next(iter(test_loader))[0].to(device)
test_batch_flat = test_batch.view(test_batch.size(0), -1)

# Inference
with torch.no_grad():
    recon_batch, _, _ = model(test_batch_flat)

# Reshape for visualization
recon_batch = recon_batch.view(-1, 1, 28, 28)
original = test_batch.view(-1, 1, 28, 28)

# Concatenate and save
comparison = torch.cat([original, recon_batch])
save_image(comparison, "inference_reconstruction.png", nrow=8)

100%|██████████| 26.4M/26.4M [00:01<00:00, 13.4MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 201kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.72MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 22.4MB/s]
